<a href="https://colab.research.google.com/github/ZA3karia/course-articles-videos-recomendation/blob/master/webscraping_indeed_jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 18 14:47:12 2019

@author: ZAZA
"""

import re
import json
from bs4 import BeautifulSoup
from time import sleep
import requests
# be kind to the server - set num seconds to delay prior to
# fetching each URL
FETCH_DELAY_SECONDS = 0.01 # or whatever value you're comfortable with
#from newspaper import Article
from pandas import DataFrame


def get_soup(url):
   sleep(FETCH_DELAY_SECONDS)
   req = requests.get(url)
   soup = BeautifulSoup(req.text, 'html.parser') 
   return soup



def grab_job_links(soup):
    """
    Grab all non-sponsored job posting links from a Indeed search result page using the given soup object
    
    Parameters:
        soup: the soup object corresponding to a search result page
                e.g. https://ca.indeed.com/jobs?q=data+scientist&l=Toronto&start=20
    
    Returns:
        urls: a python list of job posting urls
    
    """
    i=0
    urls = []
    for div in soup.find_all('div', {'class': 'jobsearch-SerpJobCard unifiedRow row result'}):
      for link in div.find_all('div',{'class': 'title'}):
        i+=1
        try:
          partial_url = link.a.get('href')
          url = 'https://www.indeed.com' + partial_url  
          if i%2==0:
            urls.append(url)
            print(i/2)
        except:
          continue
    
    return urls



def get_urls(query, num_pages, location):
    """
    Get all the job posting URLs resulted from a specific search.
    
    Parameters:
        query: job title to query
        num_pages: number of pages needed
        location: city to search in
    
    Returns:
        urls: a list of job posting URL's (when num_pages valid)
    """
    # We always need the first page
    base_url = 'https://www.indeed.com/jobs?q={}&l={}'.format(query, location)
    soup = get_soup(base_url)
    urls = []
    urls.append( grab_job_links(soup))
    # Additional work is needed when more than 1 page is requested
    if num_pages >= 2:
        # Start loop from page 2 since page 1 has been dealt with above
        for i in range(2, num_pages+1):
            num = (i-1) * 10
            base_url = 'https://www.indeed.com/jobs?q={}&l={}&start={}'.format(query, location, num)
            print(base_url)
            
            soup = get_soup(base_url)
            # We always combine the results back to the list
            #print("current urls state :" + grab_job_links(soup)[0])
            urls.append(grab_job_links(soup))
                
            """
            
            try:
                soup = get_soup(base_url)
                # We always combine the results back to the list
                print("current urls state :" + grab_job_links(soup))
                urls.append(grab_job_links(soup))
                
            except:
                continue
            """

    # Check to ensure the number of urls gotten is correct
    #assert len(urls) == num_pages * 10, "There are missing job links, check code!"

    return urls     



def get_posting(url):
    """
    Get the text portion including both title and job description of the job posting from a given url
    
    Parameters:
        url: The job posting link
        
    Returns:
        title: the job title (if "data scientist" is in the title)
        posting: the job posting content    
    """
    # Get the url content as BS object
    soup = get_soup(url)
    
    # The job title is held in the h3 tag
    title = soup.find(name='h3').getText().lower()
    posting = soup.find(name='div', attrs={'class': "jobsearch-jobDescriptionText"}).get_text()

    return title, posting.lower()

    #if 'data scientist' in title:  # We'll proceed to grab the job posting text if the title is correct
        # All the text info is contained in the div element with the below class, extract the text.
        #posting = soup.find(name='div', attrs={'class': "jobsearch-JobComponent"}).get_text()
        #return title, posting.lower()
    #else:
        #return False
    
        # Get rid of numbers and symbols other than given
        #text = re.sub("[^a-zA-Z'+#&]", " ", text)
        # Convert to lower case and split to list and then set
        #text = text.lower().strip()
    
        #return text
      
"""
def get_art(url):
  article = Article(url)
  article.download()
  article.parse()
  article.nlp()
  title = article.title
  text = article.text
  return title,text

"""
def get_data(query, num_pages, location='usa'):
    """
    Get all the job posting data and save in a json file using below structure:
    
    {<count>: {'title': ..., 'posting':..., 'url':...}...}
    
    The json file name has this format: ""<query>.json"
    
    Parameters:
        query: Indeed query keyword such as 'Data Scientist'
        num_pages: Number of search results needed
        location: location to search for
    
    Returns:
        postings_dict: Python dict including all posting data
    
    """
    # Convert the queried title to Indeed format
    query = '+'.join(query.lower().split())
    
    #postings_dict = {}
    tab_url = []
    tab_titles = []
    tab_post = []
    urls = get_urls(query, num_pages, location)
    print("---------------------------------------------part1-------------------------------------------------")
    #  Continue only if the requested number of pages is valid (when invalid, a number is returned instead of list)
    i=0
    print("----------------------------urls:" )
    print(urls)
    for page_urls in urls:
      
      print("page urls :" )
      print(page_urls)
      for url in page_urls:
        print("url")
        print( url)
        try:
            title, posting = get_posting(url)
            #title, posting = get_art(url)
            tab_url.append(url)
            tab_titles.append(title)
            tab_post.append(posting)
            #postings_dict[i] = {}
            #postings_dict[i]['title'], postings_dict[i]['posting'], postings_dict[i]['url'] = title, posting, url
            postings_table.append(post)
            #i+=1
        except: 
            continue
        """
        
        percent = (i+1) / num_urls
        # Print the progress the "end" arg keeps the message in the same line 
        print("Progress: {:2.0f}%".format(100*percent), end='\r')

        """
    posting_dict = {'title' : tab_titles,
                    'url' : tab_url,
                    'post' : tab_post
    }
    # Save the dict as json file
    file_name = query.replace('+', '_') + '.json'
    with open(file_name, 'w') as f:
        json.dump(posting_dict, f)

    print('All {} postings have been scraped and saved!'.format(i))    
    
    df = DataFrame(posting_dict, columns = ['title','url','post'])
    file_name = query.replace('+', '_') + '.csv'
    df.to_csv( file_name)
    return posting_dict,df



# If script is run directly, we'll take input from the user
if __name__ == "__main__":
    
    query = input("Please enter the title to scrape data for: \n").lower()
    
    while True:
        num_pages = input("Please enter the number of pages needed (integer only): \n")
        try:
            num_pages = int(num_pages)
            break
        except:
            print("Invalid number of pages! Please try again.")
    get_data(query, num_pages, location='usa')
